In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
#from PIL import Image
import pandas as pd # conda install pytables
import os

In [260]:
Data_dir = r'C:\Users\Sanna\Desktop\cricket_femaleandmale-Sanna-2023-01-03\videos'
test_h5 = "cricketW_spotato-open-close_food2DLC_resnet50_cricket_femaleandmaleJan3shuffle1_500000.csv"
ROI_dir = r'C:\Users\Sanna\Desktop\cricket_femaleandmale-Sanna-2023-01-03\roi-coords'

head = 0
thorax = 1

In [261]:
def read_dlc_csv(file_path):
    """
    read .h5 files and return pandas dataframe and a list of bodyparts
    """
    df = pd.read_csv(file_path)
    
    body_parts = np.array(df)[0]
    body_parts = list(body_parts)
    body_parts.remove('bodyparts')
    
    columns = np.array(df)[1]
    data = np.array(df)[2:]
    new_df = pd.DataFrame(data=data, columns=columns)
    new_df = new_df.drop(['coords'], axis=1)
    
    return new_df, body_parts

In [262]:
dlc_data, bodyparts = read_dlc_csv(os.path.join(Data_dir, test_h5))

C:\Users\Sanna\AppData\Local\Temp\ipykernel_11968\1762987095.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [263]:
def convert_dataframe_to_array_and_return_column_info(df, bodyparts):
    column_names = [i+e for (i, e) in zip(bodyparts, list(df.columns))]
    column_index = dict([(key, value) for key, value in zip(column_names, range(len(column_names)))])
    return np.array(df).T, column_index

In [264]:
dlc_data_array, column_dict = convert_dataframe_to_array_and_return_column_info(dlc_data, bodyparts)

In [265]:
def read_csv(file_path):
    df = pd.read_csv(file_path)
    df = df.drop(['Unnamed: 0'], axis=1)
    return df

In [266]:
ROI_meta = read_csv(os.path.join(ROI_dir, test_h5))

In [267]:
def inside(x_array, xmin, xmax, y_array, ymin, ymax):
    # where in bounds locations
    x_array = x_array.astype('float64')
    y_array = y_array.astype('float64')
    locations_inbounds = np.where((xmin < x_array) & (x_array < xmax) & (ymin < y_array) & (y_array < ymax))
    return locations_inbounds[0]

In [268]:
enclosureROI_meta = ROI_meta.iloc[0]
xmin = enclosureROI_meta['TLX'] 
xmax = enclosureROI_meta['BRX'] 
ymin = enclosureROI_meta['TLY'] 
ymax = enclosureROI_meta['BRY'] 
inbounds = inside(dlc_data_array[column_dict['thoraxx']], xmin, xmax, dlc_data_array[column_dict['thoraxy']], ymin, ymax)

In [269]:
x_array = inbounds_xcoords = dlc_data_array[column_dict['thoraxx']][inbounds].astype('float64')
y_array = inbounds_ycoords = dlc_data_array[column_dict['thoraxy']][inbounds].astype('float64') 

In [270]:
data_list = []
for i in range(len(ROI_meta['ROI'])-1):
    meta = ROI_meta.iloc[i+1]
    xmin = meta['TLX'] 
    xmax = meta['BRX'] 
    ymin = meta['TLY'] 
    ymax = meta['BRY']
    column_name = meta['ROI']
    in_ROI = inside(inbounds_xcoords, xmin, xmax, inbounds_ycoords, ymin, ymax)
    in_ROI_percentage = len(in_ROI)/len(inbounds)*100
    data_list.append([column_name, in_ROI_percentage])

In [271]:
xminlist=[]
xmaxlist=[]
yminlist=[]
ymaxlist=[]
for i in range(len(ROI_meta['ROI'])-1):
    meta = ROI_meta.iloc[i+1]
    xminlist.append(meta['TLX'])
    xmaxlist.append(meta['BRX'])
    yminlist.append(meta['TLY'])
    ymaxlist.append(meta['BRY'])

In [272]:
processed_data = np.array(data_list)
columns = processed_data.T[0]
data = processed_data.T[1]

In [273]:
processed_data

array([['shelter', '0.20083996616390717'],
       ['control1_box', '1.411321240063121'],
       ['control1_open', '0.0'],
       ['control2_box', '0.08459023205425648'],
       ['food1_box', '0.16522302635158867'],
       ['food2_box', '2.101894128646408'],
       ['food2_open', '0.046005213924244746']], dtype='<U32')

In [231]:
# Validation 
# Define the regions of interest
regions_of_interest = ["tube", "water_box", "control_box", "nectar1_box", "nectar2_box", "enclosure"]

# Initialize empty lists to store the frames within each region
tube_frames = []
water_box_frames = []
control_box_frames = []
nectar1_box_frames = []
nectar2_box_frames = []
enclosure_frames = []

# Iterate over the ROI_meta dataframe
for i in range(len(ROI_meta)):
    roi = ROI_meta.iloc[i]
    xmin = roi['TLX']
    xmax = roi['BRX']
    ymin = roi['TLY']
    ymax = roi['BRY']
    if roi['ROI'] in regions_of_interest:
        frames = inside(dlc_data_array[column_dict['thoraxx']], xmin, xmax, dlc_data_array[column_dict['thoraxy']], ymin, ymax)
        if roi['ROI'] == "tube":
            tube_frames.append(frames)
        elif roi['ROI'] == "water_box":
            water_box_frames.append(frames)
        elif roi['ROI'] == "control_box":
            control_box_frames.append(frames)
        elif roi['ROI'] == "nectar1_box":
            nectar1_box_frames.append(frames)
        elif roi['ROI'] == "nectar2_box":
            nectar2_box_frames.append(frames)
        elif roi['ROI'] == "enclosure":
            enclosure_frames.append(frames)

# Flatten the lists to get a list of all frames within each region
tube_frames = [item for sublist in tube_frames for item in sublist]
water_box_frames = [item for sublist in water_box_frames for item in sublist]
control_box_frames = [item for sublist in control_box_frames for item in sublist]
nectar1_box_frames = [item for sublist in nectar1_box_frames for item in sublist]
nectar2_box_frames = [item for sublist in nectar2_box_frames for item in sublist]
enclosure_frames = [item for sublist in enclosure_frames for item in sublist]

# Calculate the percentage of data entries within each region
tube_percent = len(tube_frames) / len(inbounds)
water_box_percent = len(water_box_frames) / len(inbounds)
control_box_percent = len(control_box_frames) / len(inbounds)
nectar1_box_percent = len(nectar1_box_frames) / len(inbounds)
nectar2_box_percent = len(nectar2_box_frames) / len(inbounds)
enclosure_percent = len(enclosure_frames) / len(inbounds)

# Check if the sum of ROI % occupation equals 1, i.e., 100% of occupation points are accounted for. 
percent_sum = tube_percent + water_box_percent + control_box_percent + nectar1_box_percent + nectar2_box_percent

if percent_sum + enclosure_percent == 1:
    print("These results are valid. The percentage of data entries within the enclosure, but not within the 'tube', 'water_box', 'control_box', 'nectar1_box', or 'nectar2_box' regions is equal to 1 when added to the percentage of data entries within the 'tube', 'water_box', 'control_box', 'nectar1_box', or 'nectar2_box' regions.")
else:
    print("The percentage of data entries within the enclosure, but not within the 'tube', 'water_box', 'control_box', 'nectar1_box', or 'nectar2_box' regions does not equal 1 when added to the percentage of data entries within the 'tube', 'water_box', 'control_box', 'nectar1_box', or 'nectar2_box' regions.")

These results are valid. The percentage of data entries within the enclosure, but not within the 'tube', 'water_box', 'control_box', 'nectar1_box', or 'nectar2_box' regions is equal to 1 when added to the percentage of data entries within the 'tube', 'water_box', 'control_box', 'nectar1_box', or 'nectar2_box' regions.
